# Stable Diffusion Training / Inference Tutorial

### Note:
Currently, this notebook must be run in a NeMo container (> 24.01). An example command to launch the container:

```
docker run --gpus all -it --rm -v <your_nemo_dir>:/opt/NeMo --shm-size=8g \
     -p 8888:8888 --ulimit memlock=-1 --ulimit \
      stack=67108864 <your_nemo_container>
```

## Introduction

This notebook illustrates how to train and perform inference using Stable Diffusion with the NeMo Toolkit. For the sake of brevity, we've chosen to use Stable Diffusion as an example to demonstrate the foundational process of training and inferencing with Text2Img models. However, you can apply the same approach to other foundational Text2Img models, such as Imagen.

The implementation of Stable Diffusion is based on [High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752).

This tutorial will guide you through the following topics:

1. Training a Stable Diffusion model.
2. Performing inference with the trained model.

## Datasets

Please refer to [Dataset Tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/multimodal/Multimodal%20Data%20Preparation.ipynb) for how to prepare a training dataset for Stable diffusion.

For a pre-cached Stable Diffusion dataset, each webdataset tar file should, at a minimum, include the pickle files that store the pre-cached image and text features:

```
t0_r0_0.tar
|---- 0000.pickle
|---- 0001.pickle
...
```

For non-precached Stable Diffusion dataset, each webdataset tar file should contain the raw texts and corresponding images:

```
t0_r0_0.tar
|---- 0000.jpg
|---- 0000.txt
|---- 0001.jpg
|---- 0001.txt
...
```

## Encoders Preparation

Depending on whether you precache the dataset, you might also need to first download the image and/or text encoders.

### Option 1: Training on Non-Precached Dataset (Use Encoders During Training)

#### A. Prepare VAE
To download the default VAE for Stable Diffusion:


In [ ]:
! wget https://huggingface.co/CompVis/stable-diffusion-v1-4/resolve/main/vae/diffusion_pytorch_model.bin
! mkdir -p /ckpts
! mv diffusion_pytorch_model.bin /ckpts/vae.bin

The above command will download the default VAE weights from HuggingFace and save it to `/ckpts/vae.bin`.

**Note**: if you want to customize the saved location, make sure it is also reflected in your training config.
#### B. Prepare Text Encoder
For the text encoder used in Stable Diffusion, you can either use [HuggingFace CLIP ViT-L/14 model](https://huggingface.co/openai/clip-vit-large-patch14) or use NeMo's CLIP-ViT. NeMo Stable Diffusion also supports native CLIP ViT model trained in NeMo Framework.

Make sure the following settings are used in `cond_stage_config`:

```
  cond_stage_config:
    # For compatibility with the previous version that uses HuggingFace CLIP model
    _target_: nemo.collections.multimodal.modules.stable_diffusion.encoders.modules.FrozenCLIPEmbedder
    version: openai/clip-vit-large-patch14
    device: cuda
    max_length: 77
    capture_cudagraph_i rs: ${model.capture_cudagraph_ters}
```

Alternatively, you can use the CLIP model in `.nemo` format . This can be achieved by using the provided NeMo script to download and convert the CLIP model via the following command:

In [ ]:
! python /opt/NeMo/examples/multimodal/vision_language_foundation/clip/convert_external_clip_to_nemo.py \
        --arch ViT-L-14 \
        --version openai \
        --hparams_file /opt/NeMo/examples/multimodal/vision_language_foundation/clip/conf/megatron_clip_VIT-L-14.yaml \
        --nemo_file /ckpts/openai.nemo

When using `.nemo` ViT model, you can use the default `cond_stage_config`:

```
  cond_stage_config:
    _target_: nemo.collections.multimodal.modules.stable_diffusion.encoders.modules.FrozenMegatronCLIPEmbedder
    restore_from_path: /ckpts/openai.nemo
    device: cuda
    freeze: True
    layer: "last"
```


### Option 2: Training on Precached Dataset (Training UNet Only)

When using precached dataset (please refer to the [Dataset Tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/multimodal/Multimodal%20Data%20Preparation.ipynb) for details), every text feature and image feature are stored as key-value pairs in `.pickle` file:

```
{
 image_key: torch.Tensor(),
 text_key: torch.Tensor(),
}
```

Make sure in the training config, `cond_stage_key` is associated with `text_key` and `first_stage_key` is associated with `image_key`.


## Model Config Setup

Now we will begin setting up the config file needed for Stable Diffusion training. We will use [sd_train.yaml]() as the template.

1. Modify `model.data.train.dataset_path` so that it has all the webdataset info files you want to train on
2. Modify `model.data.webdataset.local_root_path` to point to your dataset path
3. Make sure VAE path `model.first_stage_config.from_pretrained` is adjusted if using non-precached dataset
4. Make sure the text encoder config is correct (detailed above)
5. Configure `exp_manager.exp_dir` for experiment save directory
6. Configure `exp_manager.wandb_logger_kwargs` and/or `exp_manager.create_tensorboard_logger` if needed

**Note**: Please refer to NeMo Toolkit Developer Guide's Stable Diffusion page for more details on in-depth customizations, including all available optimizations.

## Training

Once everything is set up, training stable diffusion is as simple as running:


In [ ]:
! torchrun /opt/NeMo/examples/multimodal/text_to_image/stable_diffusion/sd_train.py trainer.max_steps=100 model.data.synthetic_data=True

Intermediate checkpoints (during training) and final checkpoint will be saved to `exp_manager.exp_dir` folder. Note that here we use synthetic data for demo purpose.

## Inference

Stable Diffusion inference needs a trained NeMo Stable Diffusion checkpoint, along with both the image encoder (VAE) and text encoder (CLIP). The checkpoint can be either a fully trained `.nemo` checkpoint or an intermediate checkpoint from training (typically in `.ckpt` format). Both `.nemo` and `.ckpt` checkpoints can be used for inference. For information on downloading the encoders, please refer to the previous section.

### Inference Config Setup

Now we will begin setting up the config file needed for Stable Diffusion inference. We will use [sd_infer.yaml]() as the template.

We generally use [Classifier Free Guidance](https://arxiv.org/abs/2207.12598) for better visual quality, which can be set at `infer.unconditional_guidance_scale`.

NeMo Stable Diffusion supports multiple samplers. Please refer to the developer guide for more details. Samplers can be set at `infer.sampler_type`.

Inference supports a batch of text prompts, which can be set at `infer.prompts`. One can also generate a configurable number of images per prompt by setting `infer.num_images_per_prompt`. Generated images will be saved to `infer.out_path`.

You will also need to set the model checkpoint path at `model.restore_from_path`.

### Running the Inference

Once everything is set up, Stable Diffusion inference is as simple as running:

In [ ]:
! torchrun  /opt/NeMo/examples/multimodal/text_to_image/stable_diffusion/sd_infer.py model.restore_from_path='/opt/NeMo/tutorials/multimodal/nemo_experiments/stable-diffusion-train/checkpoints/stable-diffusion-train.nemo'